In [1]:
%load_ext autoreload
%autoreload 2

import anndata as ad
import matplotlib.pyplot as plt
from mmd_wrapper import mmd_combine
from mmd_wrapper.utilities import alignment_visualize, pairwise_error
import numpy as np
import pandas as pd

from commando import ComManDo

2021-10-30 17:12:45.950189: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-30 17:12:45.950226: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Instructions for updating:
non-resource variables are not supported in the long term


# Read Data

In [2]:
data_mod1 = ad.read_h5ad('OPSCA/openproblems_bmmc_multiome_phase1/openproblems_bmmc_multiome_phase1.censor_dataset.output_mod1.h5ad')
data_mod2 = ad.read_h5ad('OPSCA/openproblems_bmmc_multiome_phase1/openproblems_bmmc_multiome_phase1.censor_dataset.output_mod2.h5ad')

In [5]:
X1 = data_mod1.X[:10000]
X2 = data_mod2.X[:10000]

# Run Mapping

In [6]:
commando_out = (
    ComManDo(
        distance_mode='euclidean',
        epoch_pd=1000,
        two_step_pd_large=2000,
        log_pd=1000,
        two_step_num_partitions=100,
        #two_step_omit_large=True,
        #prime_dual_verbose_timer=True,
    )
    .fit_transform([X1, X2])
)

use random seed: 666
Shape of Raw data
Dataset 0: (10000, 13431)
Dataset 1: (10000, 116490)
Device: cpu
---------------------------------
Find correspondence between Dataset 1 and Dataset 1
Calculating intra-group F #10
epoch:[1000/1000] err:0.5302 alpha:0.9691
Calculating intra-group F #20
epoch:[1000/1000] err:0.5155 alpha:0.9690
Calculating intra-group F #30
epoch:[1000/1000] err:0.5180 alpha:0.9693
Calculating intra-group F #40
epoch:[1000/1000] err:0.5185 alpha:0.9686
Calculating intra-group F #50
epoch:[1000/1000] err:0.5370 alpha:0.9696
Calculating intra-group F #60
epoch:[1000/1000] err:0.5224 alpha:0.9698
Calculating intra-group F #70
epoch:[1000/1000] err:0.5178 alpha:0.9690
Calculating intra-group F #80
epoch:[1000/1000] err:0.5277 alpha:0.9684
Calculating intra-group F #90
epoch:[1000/1000] err:0.5357 alpha:0.9693
Calculating intra-group F #100
epoch:[1000/1000] err:0.5112 alpha:0.9694
Constructing large F
Calculating inter-group F
epoch:[1000/2000] err:4.3456 alpha:0.8990


KeyboardInterrupt: 

# Evaluate

In [ ]:
print('ComManDo Pairwise Error: ' + str(pairwise_error(*commando_out, normalize_by_feature=True)))

plt.subplots(figsize=(12, 6))
plt.subplot(1, 2, 1)
alignment_visualize(*commando_out, alg_name='ComManDo')

plt.subplot(1, 2, 2)
plt.axis([-.1, .1, -.1, .1])
alignment_visualize(*commando_out, alg_name='ComManDo (Zoomed)')